In [1]:
using LowLevelFEM

[ Info: Precompiling LowLevelFEM [6171b9fb-adbf-4751-adb9-5faded75de07] (cache misses: include_dependency fsize change (2), wrong dep version loaded (2), incompatible header (14))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


In [2]:
gmsh.initialize()
gmsh.open("rectangle.geo")

mat = Material("rect")
prob = Problem([mat], type=:General, dim=2, fdim=2)
prob2 = Problem([mat], type=:PlaneStrain)

Info    : Reading 'rectangle.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000619281s, CPU 0.00062s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Transfinite)
Info    : Done meshing 2D (Wall 0.000120375s, CPU 0.000121s)
Info    : 4 nodes 9 elements
Info    : Done reading 'rectangle.geo'


Problem("rectangle", :PlaneStrain, 2, 2, Material[Material("rect", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 4, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [3]:
μ = mat.μ
λ = mat.λ

115384.61538461536

In [4]:
K = stiffnessMatrixPoisson(prob, coefficient=2μ) + gradDivMatrix(prob, coefficient=λ) - curlCurlMatrix(prob, coefficient=μ)

sparse([1, 2, 3, 4, 5, 6, 7, 8, 1, 2  …  7, 8, 1, 2, 3, 4, 5, 6, 7, 8], [1, 1, 1, 1, 1, 1, 1, 1, 2, 2  …  7, 7, 8, 8, 8, 8, 8, 8, 8, 8], [96153.84615384616, 48076.92307692308, -19230.76923076922, 9615.384615384613, -48076.92307692309, -48076.92307692308, -28846.15384615385, -9615.384615384613, 48076.92307692308, 192307.6923076923  …  96153.84615384613, -48076.92307692308, -9615.384615384613, -173076.92307692303, 48076.92307692308, -96153.84615384617, 9615.384615384613, 76923.07692307692, -48076.92307692308, 192307.69230769228], 8, 8)

In [5]:
K2 = stiffnessMatrix(prob2)

sparse([1, 2, 3, 4, 5, 6, 7, 8, 1, 2  …  7, 8, 1, 2, 3, 4, 5, 6, 7, 8], [1, 1, 1, 1, 1, 1, 1, 1, 2, 2  …  7, 7, 8, 8, 8, 8, 8, 8, 8, 8], [96153.84615384616, 48076.92307692308, -19230.769230769234, 9615.384615384615, -48076.923076923085, -48076.92307692308, -28846.153846153844, -9615.384615384615, 48076.92307692308, 192307.6923076923  …  96153.84615384616, -48076.92307692308, -9615.384615384615, -173076.9230769231, 48076.92307692308, -96153.84615384617, 9615.384615384617, 76923.07692307694, -48076.92307692308, 192307.6923076923], 8, 8)

In [6]:
bcB = BoundaryCondition("bottom", uy=0)
bcL = BoundaryCondition("left", ux=0)

load = BoundaryCondition("right", fx=1)

BoundaryCondition("right", nothing, nothing, nothing, nothing, nothing, nothing, 1, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [7]:
f = loadVector(prob, [load])

VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [8]:
u = solveDisplacement(K, f, support=[bcB, bcL])

VectorField
[0.0; 0.0; … ; 0.0; -9.750000000000003e-6;;]

In [9]:
u2 = solveDisplacement(K2, f, support=[bcB, bcL])

VectorField
[0.0; 0.0; … ; 0.0; -9.75e-6;;]

In [10]:
elsupp = BoundaryCondition("left", kx=1, ky=1)
C = elasticSupportMatrix(prob, [elsupp])
u3 = solveDisplacement(K + C, f)

VectorField
[0.9999999999837679; 4.874984816347186e-6; … ; 1.000000000029105; -4.874984816559898e-6;;]

In [11]:
BoundaryConditionFields(elsupp)

(:phName, :kx, :ky)

In [12]:
K.A

8×8 SparseArrays.SparseMatrixCSC{Float64, Int64} with 64 stored entries:
  96153.8    48076.9        -19230.8   …  -28846.2    -9615.38
  48076.9        1.92308e5   -9615.38       9615.38      -1.73077e5
 -19230.8    -9615.38        96153.8      -48076.9    48076.9
   9615.38   76923.1        -48076.9       48076.9   -96153.8
 -48076.9   -48076.9        -28846.2      -19230.8     9615.38
 -48076.9   -96153.8          9615.38  …   -9615.38   76923.1
 -28846.2     9615.38       -48076.9       96153.8   -48076.9
  -9615.38      -1.73077e5   48076.9      -48076.9        1.92308e5

In [13]:
elasticSupportMatrix(prob, [elsupp]).A

8×8 SparseArrays.SparseMatrixCSC{Float64, Int64} with 8 stored entries:
 1.66667    ⋅         ⋅    ⋅    ⋅    ⋅   0.833333   ⋅ 
  ⋅        1.66667    ⋅    ⋅    ⋅    ⋅    ⋅        0.833333
  ⋅         ⋅         ⋅    ⋅    ⋅    ⋅    ⋅         ⋅ 
  ⋅         ⋅         ⋅    ⋅    ⋅    ⋅    ⋅         ⋅ 
  ⋅         ⋅         ⋅    ⋅    ⋅    ⋅    ⋅         ⋅ 
  ⋅         ⋅         ⋅    ⋅    ⋅    ⋅    ⋅         ⋅ 
 0.833333   ⋅         ⋅    ⋅    ⋅    ⋅   1.66667    ⋅ 
  ⋅        0.833333   ⋅    ⋅    ⋅    ⋅    ⋅        1.66667

In [14]:
showDoFResults(u, :ux, name="ux")
showDoFResults(u, :uy, name="uy")

showDoFResults(u2, :ux, name="u2x")
showDoFResults(u2, :uy, name="u2y")

showDoFResults(u3, :ux, name="u2x")
showDoFResults(u3, :uy, name="u2y")

6

In [15]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [16]:
gmsh.finalize()